This Notebook contains the code same code from _main.ipynb_ but in a more clean format along with MLFlow commands

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error, root_mean_squared_error

import pickle

In [2]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("NYC-Taxi_Trip")

<Experiment: artifact_location='file:///d:/Code/ML Zoomcamp/Personal Progress/mlruns/1', creation_time=1748762907485, experiment_id='1', last_update_time=1748762907485, lifecycle_stage='active', name='NYC-Taxi_Trip', tags={}>

## Preprocessing Data

In [3]:
def read_parameter(filename):
    df = pd.read_parquet(filename)
    
    df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
    df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[((df.duration >= 0) & (df.duration <= 20))]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df 

In [4]:
df_train = read_parameter('./data/yellow_tripdata_2024-01.parquet')
df_val = read_parameter('./data/yellow_tripdata_2024-02.parquet')

In [5]:
df_train.head()


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,duration
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1.0,1.72,1.0,N,186,79,2,17.7,1.0,0.5,0.00,0.0,1.0,22.70,2.5,0.0,19.800000
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.0,1.80,1.0,N,140,236,1,10.0,3.5,0.5,3.75,0.0,1.0,18.75,2.5,0.0,6.600000
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1.0,4.70,1.0,N,236,79,1,23.3,3.5,0.5,3.00,0.0,1.0,31.30,2.5,0.0,17.916667
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1.0,1.40,1.0,N,79,211,1,10.0,3.5,0.5,2.00,0.0,1.0,17.00,2.5,0.0,8.300000
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1.0,0.80,1.0,N,211,148,1,7.9,3.5,0.5,3.20,0.0,1.0,16.10,2.5,0.0,6.100000


In [6]:
len(df_train), len(df_val)

(2312213, 2311724)

## Vectorizing Data

In [8]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dict = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [9]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

## Training with ML Algos

1. Linear Regression

In [9]:
with mlflow.start_run():
    mlflow.set_tag(key="Model", value="Linear Regression")
    
    mlflow.log_param("train_data_path", "./data/yellow_tripdata_2024-01.parquet")
    mlflow.log_param("valid_data_path", "./data/yellow_tripdata_2024-02.parquet")
    mlflow.log_param("Alpha", "NA")
    
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    
    y_pred = lr.predict(X_val)

    rmse = root_mean_squared_error(y_val, y_pred)
    print("RMSE: ", rmse)
    
    mlflow.log_metric("RMSE", rmse)

RMSE:  4.58574464535479


Lasso Regression

In [10]:
with mlflow.start_run():
    mlflow.set_tag(key="Model", value="Lasso Regression")
    
    mlflow.log_param("train_data_path", "./data/yellow_tripdata_2024-01.parquet")
    mlflow.log_param("valid_data_path", "./data/yellow_tripdata_2024-02.parquet")
    
    alpha = 0.001
    mlflow.log_param("Alpha", alpha)
    
    lr = Lasso(alpha=alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)

    rmse = root_mean_squared_error(y_val, y_pred)
    print("RMSE: ", rmse)
    mlflow.log_metric("RMSE", rmse)

RMSE:  4.601672367208853


Ridge Regression

In [15]:
with mlflow.start_run():
    mlflow.set_tag(key="Model", value="Ridge Regression")
    
    mlflow.log_param("train_data_path", "./data/yellow_tripdata_2024-01.parquet")
    mlflow.log_param("valid_data_path", "./data/yellow_tripdata_2024-02.parquet")
    mlflow.log_param("Alpha", "NA")
    
    lr = Ridge()
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)

    rmse = root_mean_squared_error(y_val, y_pred)
    print("RMSE: ", rmse)
    # with open('models/ridge_reg.bin', 'wb') as f_out:
    #     pickle.dump((dv, lr), f_out)
        
    mlflow.log_metric("RMSE", rmse)
    mlflow.log_artifact(local_path="./models/ridge_reg.bin", artifact_path="models_pickle")

RMSE:  4.583881728277053


In [18]:
from mlflow.tracking import MlflowClient

run_id = "d043b140c2584a099b62b577a3ef78b0"
client = MlflowClient()

artifact_path = "models_pickle/ridge_reg.bin"
local_path = client.download_artifacts(run_id=run_id, path=artifact_path, dst_path="./")

with open(local_path, "rb") as f_in:
    dv, lr = pickle.load(f_in)